In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json

In [2]:
main_url = 'https://aruba-realty.com/'
sub_links = ["property/list/houses-for-sale", "property/list/longterm-rentals",
             "property/list/vacation-rentals", "property/list/condominiums-for-sale", "property/list/commercial-for-sale",
             "property/list/property-land-for-sale", "property/list/commercial-rentals", "property/list/project-development"]

In [3]:
def join_links(main, sub):
    links  = []
    for sub in sub_links:
        url = main_url + sub
        links.append(url)
    return links
links = join_links(main_url, sub_links)

In [4]:
links

['https://aruba-realty.com/property/list/houses-for-sale',
 'https://aruba-realty.com/property/list/longterm-rentals',
 'https://aruba-realty.com/property/list/vacation-rentals',
 'https://aruba-realty.com/property/list/condominiums-for-sale',
 'https://aruba-realty.com/property/list/commercial-for-sale',
 'https://aruba-realty.com/property/list/property-land-for-sale',
 'https://aruba-realty.com/property/list/commercial-rentals',
 'https://aruba-realty.com/property/list/project-development']

In [6]:
def get_all_url(li):
    all_links = []
    for link in li:
        req = requests.get(link)
        soup = BeautifulSoup(req.content, "html.parser")
        anchors = soup.find('div', {'class': 'property-type-grid'})
        for anchor in anchors:
            links  = anchor.find_all('a', href = True)
            for link in links:
                urls = link.get('href')
                #print(urls)
                all_links.append(urls)
    return all_links
path = get_all_url(links)

In [7]:
def get_details(urls):
    property_list = []
    for link in urls:
        req = requests.get(link)
        soup = BeautifulSoup(req.content, 'html.parser')
        name = soup.find('span', class_ = "listing-heading").get('title')
        location = soup.find('div', class_ = "address").text.strip()
        try:
            property_status = soup.find('div', class_ = "page-property-details__status").text
        except AttributeError:
            property_status =None
        price = soup.find('div', class_ = "price").text.strip().split(" ")[1]
        rooms = soup.find_all("div", class_="listing-amenety")
        try:
            pool = rooms[2].text
            bedroom = rooms[0].text.split(" ")[0]
            bathroom = rooms[1].text.split(" ")[0]
        except IndexError:
            pool = None
            bedroom = None
            bathroom=  None
        link = link

        prop = {
            'name': name,
            'location': location,
            'property': property_status,
            'price': price,
            'bedrooms': bedroom,
            'bathrooms' : bathroom,
            'Pool':pool,
            'link': link
        }
        property_list.append(prop)
        print('Saving:', prop['name'])
        
    df = pd.DataFrame(property_list)
    print(df.head(20))
    df.to_csv('aruba_reality.csv', header=True, index=False)

In [ ]:
get_details(path)

Saving: CASA Distinct
